In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install rdkit-pypi
!pip install git+https://github.com/samoturk/mol2vec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 49.3 MB/s eta 0:00:00
  Cloning https://github.com/samoturk/mol2vec to /tmp/pip-req-build-7le9lxv9
  Running command git clone --filter=blob:none --quiet https://github.com/samoturk/mol2vec /tmp/pip-req-build-7le9lxv9
  Resolved https://github.com/samoturk/mol2vec to commit 850d944d5f48a58e26ed0264332b5741f72555aa
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 21.0 MB/s eta 0:00:00
  Created wheel for mol2vec: filename=mol2vec-0.1-py3-none-any.whl size=14017 sha256=3054d71d488f27a5939c1c77743974eb1b4fb11af5feb23d7025d478d8f0f3be
  Stored in directory: /tmp/pip-ephem-wheel-cache-ytkp0tm2/wheels/62/44/ea/a1bbcb466ac04cc9c230e5f172ed22a4343f0a08d69a06b168
Successfully built mol2vec


In [3]:
import sys
import os
import requests
import subprocess
import shutil
from logging import getLogger, StreamHandler, INFO
from rdkit import Chem
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score

In [14]:
import pickle
import numpy as np
import pandas as pd
from rdkit import Chem
from gensim.models import word2vec
from mol2vec.features import mol2alt_sentence, MolSentence, DfVec

# Load the trained RandomForestRegressor model
model_filename = "/content/drive/MyDrive/hacklytics/random_forest_model.pkl"
with open(model_filename, "rb") as file:
    loaded_model = pickle.load(file)

print("Random Forest Model successfully loaded!")

# Load the pre-trained Word2Vec model used for embedding
w2v_model_path = "/content/drive/MyDrive/hacklytics/Drug_Repurposing/model_300dim.pkl"
w2v_model = word2vec.Word2Vec.load(w2v_model_path)

print("Word2Vec Model successfully loaded!")

# Function to convert a single SMILES string to mol2vec embedding
def smiles_to_mol2vec(smiles, model, unseen="UNK"):
    """ Converts a SMILES string to a Mol2Vec vector using Gensim 4.0+ API """
    mol = Chem.MolFromSmiles(smiles)  # Convert to RDKit Mol object
    if mol is None:
        raise ValueError("Invalid SMILES string provided.")

    sentence = MolSentence(mol2alt_sentence(mol, 1))  # Generate sentence
    vector = []

    for word in sentence:
        if word in model.wv.key_to_index:
            vector.append(model.wv[word])
        elif unseen is not None:
            vector.append(model.wv[unseen] if unseen in model.wv.key_to_index else np.zeros(model.vector_size))

    return np.mean(vector, axis=0) if vector else np.zeros(model.vector_size)

# Example: Define your single input SMILES string
input_smiles = "CC1(C(=O)N2C(C(=O)N3CCCC3C2(O1)O)CC4=CC=CC=C4)NC(=O)C5CN(C6CC7=CNC8=CC=CC(=C78)C6=C5)C"  # Replace this with the actual SMILES input

# Convert the SMILES input to a vector
X_input = np.array([smiles_to_mol2vec(input_smiles, w2v_model)])

print(f"Processed Input Shape: {X_input.shape}")  # Should match the shape of X during training

# Make prediction
y_pred = loaded_model.predict(X_input)

print(f"Predicted vina_score: {y_pred[0]}")


Random Forest Model successfully loaded!
Word2Vec Model successfully loaded!
Processed Input Shape: (1, 100)
Predicted vina_score: -7.197414664252125
